[View in Colaboratory](https://colab.research.google.com/github/mukul-rathi/workshop-deep-learning/blob/master/ConvNet.ipynb)

# Convolutional Neural Net

This is the notebook accompanying the CNN workshop

First, let's import dependencies:

In [1]:
import numpy as np

#import the keras functions
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense
from keras.optimizers import Adam
from  keras.utils import to_categorical as OneHotEncode

#to visualise outputs
from keras.callbacks import TensorBoard
#import the MNIST dataset
from keras.datasets import mnist



Using TensorFlow backend.


Read in the data:

In [0]:
#Load in the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10
num_train_examples = x_train.shape[0]
num_test_examples = x_test.shape[0]

# reshape array so dims are (num_examples, 28, 28, 1)
x_train = np.reshape(x_train, (num_train_examples, 28,28,1))
x_test = np.reshape(x_test, (num_test_examples, 28,28,1))

# divide both arrays by 255 (to normalise from [0..255] to [0..1])
x_train= x_train / 255
x_test = x_test / 255

#one hot encode
y_train = OneHotEncode(y_train, num_classes)
y_test = OneHotEncode(y_test, num_classes)

Define the model

In [0]:
def initCNN(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1),
               activation='relu',
           input_shape=(input_shape), padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model



In [4]:
convNet = initCNN(x_train[0].shape, num_classes)
convNet.summary() #print the shape of the 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               200832    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total para

In [0]:
convNet.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [26]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


Redirecting output to ‘wget-log.5’.
Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://a691eb20.ngrok.io


In [28]:
visualisations = TensorBoard(log_dir='./logs', histogram_freq=5, batch_size=256, write_graph=True, write_grads=True, write_images=True)
visualisations.set_model(convNet)
convNet.fit(x_train, y_train, epochs=5, batch_size=256,verbose=1, validation_data=(x_test, y_test), callbacks=[visualisations])


Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 58us/step - loss: 0.0645 - acc: 0.9806 - val_loss: 0.0653 - val_acc: 0.9785
Epoch 2/5
60000/60000 [==============================] - 4s 59us/step - loss: 0.0631 - acc: 0.9810 - val_loss: 0.0807 - val_acc: 0.9739
Epoch 3/5
60000/60000 [==============================] - 4s 59us/step - loss: 0.0624 - acc: 0.9815 - val_loss: 0.0638 - val_acc: 0.9799
Epoch 4/5
60000/60000 [==============================] - 3s 57us/step - loss: 0.0599 - acc: 0.9819 - val_loss: 0.0804 - val_acc: 0.9741
Epoch 5/5
60000/60000 [==============================] - 3s 57us/step - loss: 0.0593 - acc: 0.9823 - val_loss: 0.0747 - val_acc: 0.9782


In [8]:
loss, accuracy = convNet.evaluate(x_test, y_test, verbose=0)
print("Test Accuracy: " + str(accuracy))

Test Accuracy: 0.9341


In [19]:
!cd ../..
!ls -al

total 31508
drwxr-xr-x 1 root root     4096 Oct  2 20:54 .
drwxr-xr-x 1 root root     4096 Oct  2 19:19 ..
drwxr-xr-x 4 root root     4096 Sep 28 23:11 .config
drwxr-xr-x 4 root root     4096 Oct  2 20:25 Graph
-rwxr-xr-x 1 root root 16117632 Jul 15  2017 ngrok
-rw-r--r-- 1 root root  5363700 Oct  2 20:50 ngrok-stable-linux-amd64.zip
-rw-r--r-- 1 root root  5363700 Oct  2 20:51 ngrok-stable-linux-amd64.zip.1
-rw-r--r-- 1 root root  5363700 Oct  2 20:52 ngrok-stable-linux-amd64.zip.2
drwxr-xr-x 2 root root     4096 Sep 28 23:32 sample_data
-rw-r--r-- 1 root root       48 Oct  2 20:47 url.txt
-rw-r--r-- 1 root root      822 Oct  2 20:50 wget-log
-rw-r--r-- 1 root root      827 Oct  2 20:51 wget-log.1
-rw-r--r-- 1 root root      830 Oct  2 20:52 wget-log.2


### Visualise the activations:
